In [1]:
import numpy as np
import glob

import rioxarray as rio
import xarray as xr

from natsort import natsorted 
import dask.array as da

import matplotlib.pyplot as plt
# import cartopy.crs as ccrs

# import pandas as pd
import dask

from time import time, sleep
from dask.diagnostics import ProgressBar
from dask.distributed import progress

In [2]:
import os

homedir = os.environ['HOME']
daskpath=os.path.join(homedir, "dask-worker-space-can-be-deleted")

try: 
    os.mkdir(daskpath) 
except OSError as error: 
    print(error) 

[Errno 17] File exists: '/home/kerrie/dask-worker-space-can-be-deleted'


if you start a jupyter notebook session with multiple nodes/cores

you can use local cluster below

setting a worker to multiple cores is the way to go

the default settings with more workers less threads takes much longer to compute

In [3]:
# from dask.distributed import Client,LocalCluster

# cluster=LocalCluster(n_workers=1,
#                     threads_per_worker=20)#,
# #                     memory_limit='180GB')

# client=Client(cluster)
# cluster.scale(4)
# sleep(2)
# client

In [4]:
client.close()

if you start a jupyter notebook session with only a few cores

you can use SLURM cluster

again setting a worker to multiple cores is the way to go

the default settings with more workers less threads takes much longer to compute

In [44]:
# from dask.distributed import Client
# from dask_jobqueue import SLURMCluster

# cluster = SLURMCluster(
#     cores=20,
#     processes=1,
#     memory="180GB", #40 cores x 3GB/core
#     queue="400p48h",
#     local_directory='$TMPDIR',
#     walltime="00:05:00",
#     log_directory=daskpath)

# client=Client(cluster)
# cluster.scale(jobs=9)
# sleep(10)
# client

/work/hpc/users/kerrie/UN_FAO/xr-rio-dask/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40847 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://172.25.1.51:40847/status,
Dashboard: http://172.25.1.51:40847/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.25.1.51:38629,Workers: 0
Dashboard: http://172.25.1.51:40847/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [6]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    cores=20, # actual cores (not logical) can't be more cores that are available on a single node (20 for Orion)
    processes=1, # choose 1 process, then scale to more with cluster.scale below
    memory="180GB", # cores x mem/core
    queue="400p48h", # for Orion MSU jobs (not NOAA)
    local_directory='$TMPDIR', 
    walltime="00:05:00", # set this as short as possible or you'll have to go in and kill your workers
    log_directory=daskpath)


client=Client(cluster)
# cluster.scale(jobs=9)
sleep(10)
client

/work/hpc/users/kerrie/UN_FAO/xr-rio-dask/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41840 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://172.25.1.51:41840/status,
Dashboard: http://172.25.1.51:41840/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.25.1.51:38335,Workers: 0
Dashboard: http://172.25.1.51:41840/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [48]:
sleep(10)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://172.25.1.51:40847/status,
Dashboard: http://172.25.1.51:40847/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.25.1.51:38629,Workers: 0
Dashboard: http://172.25.1.51:40847/status,Total threads: 0
Started: 1 minute ago,Total memory: 0 B


In [ ]:
# print(cluster.job_script())

In [ ]:
# client = Client(cluster)
# cluster.scale(jobs=3)  # scale to more jobs
# sleep(10)              # pause while SLURM starts up the jobs
# client

In [ ]:
client.close()

In [ ]:
# client

In [ ]:
# from dask.distributed import Client
# client=Client()
# client

In [33]:
# your notebook directory location
# work_dir='K:/projects/rgmg/climate/UN_FAO/repo_pyAEZ_data_prep/'
repo_dir='/work/hpc/users/kerrie/UN_FAO/repos/py_AEZ_data_prep/'

# your data directory location
# data_dir='C:/Users/kerrie/Documents/02_LocalData/DataDownload03152023/'
# data_dir='C:/Users/kerrie.WIN/Documents/data/DataDownload03152023/'
data_dir='/work/hpc/users/kerrie/UN_FAO/data/'

# your output data file directory
# out_dir='C:/Users/kerrie.WIN/Documents/data/pyAEZ_data_inputs_03152023/'
# out_dir='/work/hpc/users/kerrie/UN_FAO/data/gaez_nc_files/'

# maskfile='/work/hpc/users/kerrie/UN_FAO/data/orig/DataDownload03152023/ALOSmask5m_fill.rst'
maskfile='/work/hpc/users/kerrie/UN_FAO/scripts/PXV_script/ALOSmask5m_fill.rst'

# linux path of project directory (to include in file metadata)
# this one is the same for everyone
# source_dir='/gri/projects/rgmg/climate/UN_FAO/'

# input file name components
# dataset='AgERA5'
# experiment='Hist'
# year='1980'
# months=[str(x).rjust(2,'0') for x in np.arange(1,13)]
# gridsize='5m'
# file_ext_d='.pxv'
# file_exts_m=['.rst','.rdc']
# variables=['Precip','Srad','Tmax-2m','Tmin-2m','Vapr','Wind-10m','Elevation','Mask']

# some constants
# landfrac_thresh=2500
# month_int=np.arange(1,13)

year=1980

# metadata for output data files
timeattrs={'standard_name':'time','long_name':'time','axis':'T'}
time_encoding={'calendar':'standard','units':'days since 1900-01-01 00:00:00','_FillValue':None}
latattrs={'standard_name':'latitude','long_name':'latitude','units':'degrees_north','axis':'Y'}
lat_encoding={'_FillValue':None}
lonattrs={'standard_name':'longitude','long_name':'longitude','units':'degrees_east','axis':'X'}
lon_encoding={'_FillValue':None}

In [11]:
mask=xr.open_dataset(maskfile,engine='rasterio').squeeze()['band_data'] 
mask=mask.drop('band')

mask

<xarray.DataArray 'band_data' (y: 2160, x: 4320)>
[9331200 values with dtype=float32]
Coordinates:
  * x            (x) float64 -180.0 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
  * y            (y) float64 89.96 89.88 89.79 89.71 ... -89.79 -89.87 -89.96
    spatial_ref  int64 ...
Attributes:
    units:    unspecified

In [12]:
ny,nx=mask.shape

ny,nx

(2160, 4320)

In [13]:
# what is the range of values in the mask
# this can also be found in the RDC file
# it's not 0 to 1, it's 0 to 10000
np.unique(mask)

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 9.998e+03, 9.999e+03,
       1.000e+04], dtype=float32)

In [14]:
yvals=mask.y.data.astype('float32')
xvals=mask.x.data.astype('float32')
doys=(np.arange(365)+1).astype('int32')
yvals

array([ 89.958336,  89.875   ,  89.791664, ..., -89.791664, -89.875   ,
       -89.958336], dtype=float32)

In [15]:
# create an empty arr with grid and time metadata
# so we can chunk it with the metadata 
# and then easily concatenate everything

# empty_arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',yvals),'x':('x',xvals),'doy':('doy',doys)}).astype('float32')

In [16]:
# landmask_any=np.where(mask>0,1,0)
# landmask_25pct=np.where(mask>2500,1,0)

In [17]:
# list of dat files to convert
filelist=natsorted(glob.glob(data_dir+'/gaez_dat_files/global_'+str(year)+'/*'+str(year)+'*'))
filelist

['/work/hpc/users/kerrie/UN_FAO/data//gaez_dat_files/global_1980/Precip365_AgERA5_Hist_1980_5m.dat',
 '/work/hpc/users/kerrie/UN_FAO/data//gaez_dat_files/global_1980/Srad365_AgERA5_Hist_1980_5m.dat',
 '/work/hpc/users/kerrie/UN_FAO/data//gaez_dat_files/global_1980/Tmax-2m365_AgERA5_Hist_1980_5m.dat',
 '/work/hpc/users/kerrie/UN_FAO/data//gaez_dat_files/global_1980/Tmin-2m365_AgERA5_Hist_1980_5m.dat',
 '/work/hpc/users/kerrie/UN_FAO/data//gaez_dat_files/global_1980/Vapr365_AgERA5_Hist_1980_5m.dat',
 '/work/hpc/users/kerrie/UN_FAO/data//gaez_dat_files/global_1980/Wind-10m365_AgERA5_Hist_1980_5m.dat']

In [18]:
# # check out a single line
# with open (filelist[0]) as f:
#     test=f.readline()

# test

In [19]:
# get each line as a string and remove carriage returns
temp=open(filelist[0]).read().splitlines() 
# temp[0:2]

In [20]:
ilatilon=temp[0::2] # grab the lines with the lat/lons (every other line)
data=temp[1::2]  # grab the lines with the data (every other line)

# ilatilon[0:3]

In [21]:
ilat=np.array([int(i.split()[0]) for i in ilatilon]).astype('int16') # get the first number in each string as integer and put it in an array
ilon=np.array([int(i.split()[1]) for i in ilatilon]).astype('int16') # get the second number in each string as integer and put it in an array

# print(ilat.shape)
# ilat[0:3]
print(ilat.min(),ilat.max())
print(ilon.min(),ilon.max())

77 1794
1 4320


In [22]:
# string data to numpy array
# takes 30-60s to get millions of data strings into a numpy array
# makes slicing up the data super easy
data2D=np.loadtxt(data,dtype='int16')
print(data2D.min(),data2D.max())
data2D.shape

-9999 10000


(2295358, 365)

# Testing dask Futures vs dask delayed 

we want to process one entire latitude at a time (1 latitude, all longitudes, all days) so we can parallelize computations

on our 5m global grid, there are 2160 latitudes

For futures, you first fix up your lists of inputs 
- list length 2160 
- each item in a list is an np.array of info

Then, feed your inputs to the function 
- function runs 2160 times in parallel
- spits out a chunk of the global data
- the chunk it spits out has dims (y=1,x=4320,t=365)

In [ ]:
# this is our function
def build_global_data_chunk(ixs,data,y,x,t):   
    # create an array with nan at missing data grids
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')
    # loop thru longitudes to assign each grid cell data and change -9999 to nan
    if np.any(ixs):
        for i,ix in enumerate(ixs):
                arr[0:1,ix:ix+1,:]=data[i,:]
        arr=arr.where(arr>-9999)    
    return arr

In [ ]:
# prepare all input data arrays in lists of len 2160
lon_arrlist=[]    # the indexes of longitudes that have data at each latitude
data_arrlist=[]   # all the data rows grouped by latitude
yvals_arrlist=[]  # each single latitude
xvals_arrlist=[]  # all longitudes
doys_arrlist=[]   # all days

# loop thru latitudes
for iy in range(ny):
    indices=np.where(ilat==iy+1)[0]  # find which data rows apply to this latitude
    lon_arrlist.append(ilon[indices]-1)
    data_arrlist.append(data2D[indices,:])
    yvals_arrlist.append(yvals[iy:iy+1])
    xvals_arrlist.append(xvals)
    doys_arrlist.append(doys)
    

# example of what one item in each list looks like
# note: if the latitude is all ocean, there will be empty arrays for lon and data
lon_arrlist[76],data_arrlist[76],yvals_arrlist[76],xvals_arrlist[76],doys_arrlist[76]

if we compute below without pre-scattering the data to the workers, we get a warning

but the compute does complete

on 4 workers * 20 threads = 80 total threads this takes 2min 35s 

In [ ]:
# this statement sets of the parallel computing, futures is a list of pointers to where things are computing on the workers
futures=client.map(build_global_data_chunk,lon_arrlist,data_arrlist,yvals_arrlist,xvals_arrlist,doys_arrlist)

# this statement pulls down the results from all workers and puts them in a list called output
output=client.gather(futures)

# show the first item in the list output, each item is a chunk of the global data in xarray data array format
output[0]

there's also an option to pre-spread the data across the workers

on 4 workers * 20 threads = 80 total threads this also takes 2min 35s and avoids the warning

In [ ]:
# these statements pre-spread the data arrays across the workers in a round robin for each variable
scattered_lon=client.scatter(lon_arrlist)
scattered_data=client.scatter(data_arrlist)
scattered_yvals=client.scatter(yvals_arrlist)
scattered_xvals=client.scatter(xvals_arrlist)
scattered_doys=client.scatter(doys_arrlist)

# same as before but operate on the scattered data
futures=client.map(build_global_data_chunk,scattered_lon,scattered_data,scattered_yvals,scattered_xvals,scattered_doys)
output=client.gather(futures)
output[0]

the last step is to concatenate all the data output chunks together into 1 large array of global data

because we are using xarray with labeled dimensions, this is very easy

this operation is not parallelized but only takes a few seconds

In [ ]:
bigarr=xr.concat(output,dim='y')
bigarr

### we can also do the above slightly differently using dask delayed

this time instead of prepping the inputs into lists, we operate on chunks of the data inside of a loop

inside the loop we issue calls to functions that do the heavier computing tasks

we delay those functions to create a dask task graph on length 2160 and then set off all the tasks to compute

In [34]:
# similar to build_global_data_chunk function, this function builds a chunk of data for grids where data is present

@dask.delayed
def build_full_lat(ixs,data,y,x,t):
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')
    for i,ix in enumerate(ixs):
        arr[0:1,ix:ix+1,:]=data[i,:]
    
    arr=arr.where(arr>-9999)
    
    return arr

In [35]:
# similar to build_global_data_chunk function, this function builds a chunk of data for grids where data is not present

@dask.delayed
def build_empty_lat(y,x,t):
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')
    return arr

now we write a normal not-delayed loop to call the delayed functions and collect all the delayed tasks into a list called tasklist

In [41]:
tasklist=[]

# again, loop thru latitudes
for iy in range(ny):
    indices=np.where(ilat==iy+1)[0] # find which data rows apply to this latitude
    if np.any(indices):
        result=build_full_lat((ilon[indices]-1),data2D[indices,:],yvals[iy:iy+1],xvals,doys) # lazy func call
        tasklist.append(result) # collect list of resulting arrays
    else:
        result=build_empty_lat(yvals[iy:iy+1],xvals,doys)
        tasklist.append(result)

on 4 workers * 20 threads = 80 total threads this takes 2min 29s

and we just ignore the warning

In [42]:
# this statement starts the parallel computing and pulls all the results down into a list of arrays
# note: the use of *tasklist means output will be a list len 2160 of arrays, without * the output will be len 1
output=dask.compute(*tasklist)
output[0]

<xarray.DataArray (y: 1, x: 4320, doy: 365)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float16)
Coordinates:
  * y        (y) float32 89.96
  * x        (x) float32 -180.0 -179.9 -179.8 -179.7 ... 179.7 179.8 179.9 180.0
  * doy      (doy) int32 1 2 3 4 5 6 7 8 9 ... 358 359 360 361 362 363 364 365

In [ ]:
bigarr=xr.concat(output,dim='y')
bigarr

## test running by latitude, but separating out fully missing latitudes

In [ ]:
# delay all our data
# data2D=dask.persist(data2D)
# ilon=dask.persist(ilon)
# xvals=dask.persist(xvals)
# doys=dask.persist(doys)

In [ ]:
ind_lists=[]
for iy in range(ny):
    ind_lists.append(np.where(ilat==iy+1)[0])
    
ind_lists[0:3]

In [ ]:
import dask.bag as db
b=db.from_sequence(ind_lists,npartitions=ny)
b

In [ ]:
scattered_data=client.scatter(data2D,broadcast=True)
scattered_ind=client.scatter()

In [ ]:
def get_data(indices,data=None,y=None,x=None,t=None):
    data_segment=data[indices,:]
    return data_segment
    

In [ ]:
@dask.delayed
def build_full_lat(ixs,data,y,x,t):
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')
    for i,ix in enumerate(ixs):
        arr[0:1,ix:ix+1,:]=data[i,:]
    
    arr=arr.where(arr>-9999)
    
    return arr

In [ ]:
@dask.delayed
def build_empty_lat(y,x,t):
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')
    return arr

In [ ]:
tasklist=[]
for iy in range(ny):
    indices=np.where(ilat==iy+1)[0] # find which data rows apply to this latitude
    if np.any(indices):
        result=build_full_lat((ilon[indices]-1),data2D[indices,:],yvals[iy:iy+1],xvals,doys) # lazy func call
        tasklist.append(result) # collect list of resulting arrays
    else:
        result=build_empty_lat(yvals[iy:iy+1],xvals,doys)
        tasklist.append(result)

In [ ]:
# progress=progress()
# pbar.register()

len(tasklist)

In [ ]:
# # delay all our data
# data2D=dask.delayed(data2D)
# ilon=dask.delayed(ilon)
# xvals=dask.delayed(xvals)
# doys=dask.delayed(doys)

In [ ]:
# remote_data=client.scatter([data2D,ilon,xvals,doys],broadcast=True)

In [ ]:
lon_arrlist=[]
data_arrlist=[]
yvals_arrlist=[]
xvals_arrlist=[]
doys_arrlist=[]

for iy in range(ny):
    indices=np.where(ilat==iy+1)[0]
    lon_arrlist.append(ilon[indices]-1)
    data_arrlist.append(data2D[indices,:])
    yvals_arrlist.append(yvals[iy:iy+1])
    xvals_arrlist.append(xvals)
    doys_arrlist.append(doys)
    
    
lon_arrlist[0:5]    
    

In [ ]:
lon_arrlist[0],data_arrlist[0],yvals_arrlist[0],xvals_arrlist[0],doys_arrlist[0]


In [ ]:
doys_arrlist[0].size*doys_arrlist[0].itemsize*1E-6

In [ ]:

data_inputs=[lon_arrlist,data_arrlist,yvals_arrlist,xvals_arrlist,doys_arrlist]

totalsize=0
for dataset in data_inputs:
    dsize=0
    for d in dataset:
        dsize=dsize+d.size*d.itemsize*1E-6
    totalsize=totalsize+dsize
    print(dsize)
print('total =',totalsize,'MB')
    
# doys_arrlist[0].size*doys_arrlist[0].itemsize*1E-6

In [ ]:
arr_on_worker=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',yvals[0:1]),'x':('x',xvals),'doy':('doy',doys)}).astype('float16')
arr_on_worker.nbytes*1E-6

In [ ]:
scattered_data=client.scatter([lon_arrlist,data_arrlist,yvals_arrlist,xvals_arrlist,doys_arrlist])
# remote_data=client.scatter([xvals,doys],broadcast=True)

In [ ]:
scattered_data

In [ ]:
def build_global_data(ixs,data,y,x,t):
    
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')

    if np.any(ixs):
        for i,ix in enumerate(ixs):
                arr[0:1,ix:ix+1,:]=data[i,:]

        arr=arr.where(arr>-9999)    

    return arr

In [ ]:
scattered_lon=client.scatter(lon_arrlist)
scattered_data=client.scatter(data_arrlist)
scattered_yvals=client.scatter(yvals_arrlist)
scattered_xvals=client.scatter(xvals_arrlist)
scattered_doys=client.scatter(doys_arrlist)


In [ ]:
scattered_lon=client.scatter(lon_arrlist)


In [ ]:
scattered_data=client.scatter(data_arrlist)


In [ ]:
scattered_yvals=client.scatter(yvals_arrlist)
# scattered_yvals=client.scatter(yvals,broadcast=True)

In [ ]:
scattered_xvals=client.scatter(xvals_arrlist)
# scattered_xvals=client.scatter(xvals,broadcast=True)

In [ ]:
scattered_doys=client.scatter(doys_arrlist)
# scattered_doys=client.scatter(doys,broadcast=True)

In [ ]:
# futures=client.map(build_global_data,scattered_data[:][0])
futures=client.map(build_global_data,scattered_lon,scattered_data,scattered_yvals,scattered_xvals,scattered_doys)
output=client.gather(futures)
output[0]

In [ ]:
futures=client.map(build_global_data,lon_arrlist,data_arrlist,yvals_arrlist,xvals_arrlist,doys_arrlist)
output=client.gather(futures)
output[0]

In [ ]:
@dask.delayed
def build_full_lat(indices,ilon,data2D,y,x,t):
    
#     data=data2D[0]
#     data=data[indices,:]
    data=data2D[indices,:]
    
#     ixs=ilon[0]
#     ixs=ixs[indices]-1
    ixs=ilon[indices]-1
    
#     x=x[0]
#     t=t[0]
        
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')
    for i,ix in enumerate(ixs):
        arr[0:1,ix:ix+1,:]=data[i,:]
    
    arr=arr.where(arr>-9999)
    
    return arr

In [ ]:
@dask.delayed
def build_empty_lat(y,x,t):
#     x=x[0]
#     t=t[0]
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype('float16')
    return arr

In [ ]:
def build_global_data(ilat,ilon,data2D,yvals,xvals,doys):
    tasklist=[]
    for iy in range(yvals.shape[0]):
        indices=np.where(ilat==iy+1)[0] # find which data rows apply to this latitude
        if np.any(indices):
#             result=build_full_lat((ilon[indices]-1),data2D[indices,:],yvals[iy:iy+1],xvals,doys) # lazy func call
            result=build_full_lat(indices,ilon,data2D,yvals[iy:iy+1],xvals,doys) # lazy func call
            output.append(result) # collect list of resulting arrays
        else:
            result=build_empty_lat(yvals[iy:iy+1],xvals,doys)
            output.append(result)

#     arr=xr.concat(output,dim='y')
#     return arr
    return tasklist

In [ ]:
tasklist=[]
for iy in range(ny):
    indices=np.where(ilat==iy+1)[0] # find which data rows apply to this latitude
    if np.any(indices):
        result=build_full_lat(indices,ilon,data2D,yvals[iy:iy+1],xvals,doys) # lazy func call
        tasklist.append(result) # collect list of resulting arrays
    else:
        result=build_empty_lat(yvals[iy:iy+1],xvals,doys)
        tasklist.append(result)

In [ ]:
len(tasklist)

In [ ]:
# tasklist=dask.persist(tasklist)
# output=dask.compute(*tasklist)
# output[0]

# output=dask.persist(*tasklist)
# output[0]

# output=dask.persist(*tasklist)
# output=dask.compute(*output)
# output[0]

output=dask.compute(*tasklist)
output[0]

In [ ]:
# bigarr=xr.concat(output,dim='y')
# bigarr

In [ ]:
bigarr=xr.concat(output,dim='y')
bigarr

In [ ]:
dask.visualize(tasklist[0])

In [ ]:
bigarr=dask.delayed(xr.concat)(output,dim='y')
bigarr=bigarr.compute()
bigarr

In [ ]:
tasklist=build_global_data(ilat,ilon,data2D,yvals,xvals,doys,'float16')
tasklist[0:5]

In [ ]:
output=dask.compute(tasklist)
output[0]

In [ ]:
test=xr.concat(output,dim='y')
test

In [ ]:
bigarr=dask.delayed(xr.concat)(output,dim='y')

In [ ]:
# create a long list of parallel tasks
bigtasks=build_global_data(ilat,ilon,data2D,yvals,xvals,doys,'int16')
bigtasks[0]

In [ ]:
# execute all tasks in parallel
biglist=dask.compute(bigtasks)
biglist[0]

In [ ]:
bigtasks=dask.delayed(build_global_data)(ilat,ilon,data2D,yvals,xvals,doys,'int16')

In [ ]:
bigtasks

In [ ]:
bigarr=bigtasks.compute()
bigarr

In [ ]:
output=[]
for iy in range(ny):
    indices=np.where(ilat==iy+1) # find which data rows apply to this latitude
    if np.any(indices):
        result=dask.delayed(build_full_lat)((ilon[indices]-1),data2D[indices,:][0,:,:],yvals[iy:iy+1],xvals,doys,'int16') # lazy func call
        output.append(result) # collect list of resulting arrays
    else:
        result=dask.delayed(build_empty_lat)(yvals[iy:iy+1],xvals,doys,'int16')
        output.append(result)

In [ ]:
vars=[ilat,ilon,data2D,yvals,xvals,doys]

sizes=[]
for var in vars:
    varsize=var.size*var.itemsize*b_to_mb
    print(varsize)
    sizes.append(varsize)

print("total size = ",sum(sizes),'MB')



In [ ]:
b_to_mb=1E-6

iy=100
indices=np.where(ilat==iy+1) # returns a tuple
lon_input=ilon[indices]-1
data_input=data2D[indices,:][0,:,:]
y_input=yvals[iy:iy+1]
x_input=xvals
t_input=doys
arr_worker=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y_input),'x':('x',x_input),'doy':('doy',t_input)}).astype(np.int16)

vars=[lon_input,data_input,y_input,x_input,t_input]

sizes=[]
for var in vars:
    varsize=var.size*var.itemsize*b_to_mb
    print(varsize)
    sizes.append(varsize)

print(arr_worker.nbytes*b_to_mb)
print("total size = ",sum(sizes),'MB')

In [ ]:
# this takes 7.5 min on 20 cores with automated dask
# this takes xx min on 20 cores with distributed client
bigarr=dask.delayed(xr.concat)(output,dim='y') # lazy concatenate all the results together
bigdata=bigarr.compute() # now execute everything

In [ ]:
bigdata.name='prcp'
bigdata

## test running by longitude (more tasks), separating out fully missing longitudes

In [ ]:
def build_full_lon(iys,data,y,x,t,dtype):
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype(dtype)
    for i,iy in enumerate(iys):
        arr[iy:iy+1,0:1,:]=data[i,:]
    
    arr=arr.where(arr>-9999)
    
    return arr

In [ ]:
def build_empty_lon(y,x,t,dtype):
    arr=xr.DataArray(dims=['y','x','doy'],coords={'y':('y',y),'x':('x',x),'doy':('doy',t)}).astype(dtype)
    return arr

In [ ]:
output=[]
for ix in range(nx):
    indices=np.where(ilon==ix+1) # find which data rows apply to this latitude
    if np.any(indices):
        result=dask.delayed(build_full_lon)((ilat[indices]-1),data2D[indices,:][0,:,:],yvals,xvals[ix:ix+1],doys,'float32') # lazy func call
        output.append(result) # collect list of resulting arrays
    else:
        result=dask.delayed(build_empty_lon)(yvals,xvals[ix:ix+1],doys,'float32')
        output.append(result)

In [ ]:
# this takes 8 min on 20 cores
bigarr=dask.delayed(xr.concat)(output,dim='x') # lazy concatenate all the results together
bigdata=bigarr.compute() # now execute everything

## running by latitude

In [ ]:
# build a chunk of global data array by filling cells with data or nan where there is not data
# the chunk is 1 latitude, all longitudes, all times
# also replace the -9999 fill value with nan
def build_global_data(ixs,data,xrchunk):
    for i,ix in enumerate(ixs):
        xrchunk[0:1,ix:ix+1,:]=data[i,:]
    
    xrchunk=xrchunk.where(xrchunk>-9999)
    
    return xrchunk
    

In [ ]:
# now build a list of lazy compute tasks by call the function for every latitude
# note: iy is the zero based index, whereas the indexes written to file from the fortran code are 1 based
output=[]
for iy in range(ny):
    indices=np.where(ilat==iy+1) # find which data rows apply to this latitude
    
    # send the subset of data (all longitudes and days for 1 latitude at a time) to the build_global_data function
    result=dask.delayed(build_global_data)((ilon[indices]-1),data2D[indices,:][0,:,:],empty_arr[iy:iy+1,:,:]) # lazy func call
    output.append(result) # collect list of resulting arrays


In [ ]:
# this take 9.5min on 20 cores
# revised method takes 7.5 min on 20 cores
bigarr=dask.delayed(xr.concat)(output,dim='y') # lazy concatenate all the results together
# bigarr=dask.delayed(da.concatenate)(output,axis=0)
bigdata=bigarr.compute() # now execute everything

In [ ]:
# chunk dataset for writing
# bigdata=bigdata.chunk({'y': 160, 'x': 320,'doy':365 })
bigdata.name='prcp'
bigdata

In [ ]:
# metadata for output data files
timeattrs={'standard_name':'doy','long_name':'day of year','axis':'T'}
time_encoding={'_FillValue':None}
yattrs={'standard_name':'latitude','long_name':'latitude','units':'degrees_north','axis':'Y'}
y_encoding={'_FillValue':None}
xattrs={'standard_name':'longitude','long_name':'longitude','units':'degrees_east','axis':'X'}
x_encoding={'_FillValue':None}

In [ ]:
# variable/coordinate metadata
varattrs={'standard_name':'prcp','long_name':'precipitation','units':'mm day-1'}
bigdata.attrs=varattrs
bigdata['y'].attrs=yattrs
bigdata['x'].attrs=xattrs
bigdata['doy'].attrs=timeattrs
bigdata

In [ ]:
# testout=bigdata[0:320,0:640,:]
# # testout.encoding['zlib']=True
# testout.encoding['chunks']=[160,320,365]
# testout

In [ ]:
from dask.diagnostics import ProgressBar


In [ ]:
del writejob3

In [ ]:
writejob

In [ ]:
writejob=bigdata.to_netcdf(data_dir+'gaez_nc_files/prcp_'+str(year)+'_5m.nc',
            encoding={'y':y_encoding,'x':x_encoding,'doy':time_encoding,'prcp':{'zlib':True}},
            compute=False)

# with ProgressBar():
#     print('writing output nc file')
#     writejob.compute()

In [ ]:
with ProgressBar():
    print('writing output nc file')
    writejob.compute()

In [ ]:
np.save(data_dir+'test.npy',bigdata)

In [ ]:
da.to_npy_stack(data_dir+'test_npy/',bigdata,axis=0)

In [ ]:
testtest=np.load(data_dir+'test.npy')
testtest

In [ ]:
testtesttest=da.from_npy_stack(data_dir+'test_npy/')
testtesttest


In [ ]:
print(ilat.min(),ilat.max())
print(ilon.min(),ilon.max())

In [ ]:
# gdata=xr.full_like(mask,np.nan).expand_dims({'doy':np.arange(365)+1},2).copy()
# gdata=gdata
# gdata

gdata=da.empty((ny,nx,365),dtype='float32',chunks=(1,nx,365))
gdata[:]=np.nan

gdata


In [ ]:
gdata[0,0,0].compute()

In [ ]:
gdata=gdata.chunk({'y': 1, 'x': 1,'doy':365 })
gdata

In [ ]:
dgdata=gdata.to_delayed()
dgdata

In [ ]:
data2D=np.loadtxt(data)
data2D.shape

In [ ]:
# loop zero based indexes
for i,(y,x) in enumerate(zip(ilat,ilon)):
    if i%100000==0: print(i)
    gdata[y-1,x-1,:]=data2D[i,:]

In [ ]:
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

# container='/lustre/project/geospatial_tutorials/wg_2020_ws/data_science_im_rs_vSCINetGeoWS_2020.sif'
# env='py_geo'

cluster = SLURMCluster(
    cores=40,
    processes=1,
    memory="120GB", #40 cores x 3GB/core
    queue="short",
    local_directory='$TMPDIR',
    walltime="00:10:00",
    log_directory=daskpath) 

In [ ]:
gdata

In [ ]:
gdata[:,:,0].plot()

In [ ]:
gdata.min(), gdata.max()

In [ ]:
gdata=gdata.chunk({'y': 160, 'x': 320,'doy':365 })
gdata

In [ ]:
gdata.chunks


In [ ]:
gdata=gdata.where(gdata>-9999.).compute()
gdata

In [ ]:
gdata[:,:,0].plot()

In [ ]:
arr=np.array(ilatilon)
out=np.char.split(arr)
out

In [ ]:
out=np.char.split(arr).astype('integer')

In [ ]:
out

In [ ]:
ilat.shape